# utility functions

In [1]:
def add_reverse_edges(G):
    """Makes directed graph undirected by adding reverse edges."""
    V,E = G
    reverse_edges = []
    for (u,v) in E:
        if (v,u) not in E and (v,u) not in reverse_edges:
            reverse_edges.append((v,u))
    return (V, E+reverse_edges)

def make_undirected(G):
    """Makes directed graph undirected by replacing ordered edge pairs with unordered sets."""
    V,E = G
    Eu = []
    for (u,v) in E:
        if {u,v} not in Eu:
            Eu.append({u,v})
    return (V, Eu)

def is_prime(n):
    """Check if n is prime."""
    if n < 2: # 0 and 1 are not primes
        return False
    for x in range(2, int(n**0.5)+1):
        if n % x == 0:
            return False
    return True

# path

In [2]:
import collections
def path(G,s,t):
    """Check if there is a path from node s to node t in graph G."""
    V,E = G
    visited = []
    queue = collections.deque([s])
    while queue: # |\label{bfs:line-outer-loop}|
        node = queue.popleft()
        if node == t:
            return True
        if node not in visited:
            visited.append(node)
            node_neighbors = [ v for (u,v) in E if u==node ] # |\label{bfs:line-find-neighbors}|
            for neighbor in node_neighbors: # |\label{bfs:line-inner-loop}|
                if neighbor not in visited:
                    queue.append(neighbor)
    return False

In [6]:
V = [1,2,3,4,5,6,7,8]
E = [ (1,2), (2,3), (3,4), (4,5), (6,7), (7,8), (8,6) ]

G = (V,E)
print( "path from {} to {}? {}".format(4, 1, path(G, 4, 1)))
print( "path from {} to {}? {}".format(1, 4, path(G, 1, 4)))

G = add_reverse_edges(G)
print( "path from {} to {}? {}".format(4, 1, path(G, 4, 1)))
print( "path from {} to {}? {}".format(1, 4, path(G, 1, 4)))
print( "path from {} to {}? {}".format(4, 7, path(G, 4, 7)))

path from 4 to 1? False
path from 1 to 4? True
path from 4 to 1? True
path from 1 to 4? True
path from 4 to 7? False


In [ ]:
def path_verify(G,s,t,p):
    V,E = G
    n = len(V)
    if p[0] != s or p[-1] != t:
        return False
    for i in range(len(p)-1): # i=0,1,2,...,|p|-1
        if (p[i],p[i+1]) not in E:
            return False
    return True

def path_verify_exact_length(G,s,t,p):
    V,E = G
    n = len(V)
    if len(p) != n:
        return False
    idx = p.index(0)
    p_shortened = p[:idx]
    return path_verify(G,s,t,p_shortened)

In [4]:
V = [1,2,3,4,5,6,7,8]
E = [ (1,2), (2,3), (3,4), (4,5), (5,6), (6,7), (7,8), (8,6) ]
G = (V,E)
G = add_reverse_edges(G)
p = [3,4,5,6,8,7]
print(path_verify(G,3,7,p))
p = [3,4,5,6,7]
print(path_verify(G,3,7,p))
p_exact = [3,4,5,6,7,0,0,0]
print(path_verify_exact_length(G,3,7,p_exact))

True
True
True


# relatively prime

In [11]:
def gcd(x,y):
    """Euclid's algorithm for greatest common divisor."""
    while y>0:
        x = x % y
        x,y = y,x
    return x

def rel_prime(x,y):
    return gcd(x,y) == 1

In [12]:
print( gcd(24,60), rel_prime(24,60))
print( gcd(25,63), rel_prime(25,63))

12 False
1 True


# connected

In [5]:
import itertools
def connected(G):
    """Check if G is connected."""
    V,E = G
    for (s,t) in itertools.combinations(V,2):
        if not path(G,s,t):
            return False
    return True

In [14]:
V = [1,2,3,4,5,6]
E = [ (1,2), (2,3), (3,4), (4,1), (5,6) ]
G = (V,E)
G = add_reverse_edges(G)
print(connected(G))

V = [1,2,3,4,5,6]
E = [ (1,2), (2,3), (3,4), (4,1), (5,6), (2,5) ]
G = (V,E)
G = add_reverse_edges(G)
print(connected(G))

False
True


# Eulerian path

In [3]:
def degree(node, E):
    """Return degree of node."""
    return sum(1 for (u,v) in E if u==node)

def eulerian_path(G):
    """Check if G has an Eulerian path."""
    V,E = G
    num_deg_odd = 0
    V_pos = [] # nodes with positive degree
    for u in V:
        deg = degree(u, E)
        if deg % 2 == 1:
            num_deg_odd += 1
        if deg > 0:
            V_pos.append(u)
    if num_deg_odd not in [0,2]:
        return False
    G_pos = (V_pos,E)
    return connected(G_pos)

In [8]:
V = [1,2,3,4,5,6]
E = [ (1,2), (2,3), (3,4), (4,1), (5,6) ]
G = (V,E)
G = add_reverse_edges(G)
print(eulerian_path(G))

V = [1,2,3,4,5,6]
E = [ (1,2), (2,3), (3,4), (4,1), (5,1) ]
G = (V,E)
G = add_reverse_edges(G)
print(eulerian_path(G))

False
True


# Hamiltonian path verifier

In [3]:
def ham_path_verify(G,p):
    """Verify that p is a Hamiltonian path in G."""
    V,E = G
    # verify each pair of adjacent nodes in p shares an edge
    for i in range(len(p) - 1):
        if (p[i],p[i+1]) not in E:
            return False
    # verify p has |V| nodes
    if len(p) != len(V):
        return False
    # verify p has no duplicates
    if len(set(p)) != len(p):
        return False
    return True

In [8]:
V = [1,2,3,4,5]
E = [ (1,2), (2,4), (4,3), (3,5), (3,1)]
G = (V,E)

p_bad = [1,2,3,4,5]
print(ham_path_verify(G, p_bad))

p_bad2 = [1,2,4,3,1,2,4,3,5]
print(ham_path_verify(G, p_bad2))

p_bad3 = [1,2,4]
print(ham_path_verify(G, p_bad3))

p_good = [1,2,4,3,5]
print(ham_path_verify(G, p_good))

False
False
False
True


# composite verifier

In [9]:
def composite_verify(n,d):
    """Verify that d is a nontrivial divisor of n."""
    if not 1 < d < n:
        return False
    return n % d == 0

def prime_verify_bad(n,d):
    """Verify that d is a nontrivial divisor of n."""
    if not 1 < d < n:
        return True
    return n % d != 0

In [10]:
n = 15
print(composite_verify(n, 3))
print(composite_verify(n, 4))
     
n = 17
for d in range(2,n):
    print(composite_verify(17, d), end=" ")
print()

n = 18
for d in range(2,n):
    print(composite_verify(18, d), end=" ")
print()

# n = 18
# for d in range(2,n):
#     print(prime_verify_bad(18, d), end=" ")

True
False
False False False False False False False False False False False False False False False 
True True False False True False False True False False False False False False False False 


# clique verifier

In [4]:
import itertools
def clique_v(G, k, C):
    """Verify C is a k-clique in G."""
    V,E = G
    # verify C is the correct size
    if len(C) != k:
        return False
    # verify each pair of nodes in C shares an edge
    for (u,v) in itertools.combinations(C, 2):
        if (u,v) not in E:
            return False
    return True

In [5]:
V = [1,2,3,4,5,6]
E = [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4), (4,5), (5,6), (4,6)]
G = (V,E)
G = add_reverse_edges(G)

C = [1,2,3,4]
k = len(C)
print('{} is a {}-clique in G?   {}'.format(C, k, clique_v(G, k, C)))

C = [3,4,5]
print('{} is a {}-clique in G?   {}'.format(C, k, clique_v(G, k, C)))

C = [4,5,6]
print('{} is a {}-clique in G?   {}'.format(C, k, clique_v(G, k, C)))

C = [1,3,4,5]
print('{} is a {}-clique in G?   {}'.format(C, k, clique_v(G, k, C)))

[1, 2, 3, 4] is a 4-clique in G?   True
[3, 4, 5] is a 4-clique in G?   False
[4, 5, 6] is a 4-clique in G?   False
[1, 3, 4, 5] is a 4-clique in G?   False


# subset sum verifier

In [6]:
import itertools
def subset_sum_v(C, t, S):
    """Verifies that S is a subcollection of C summing to t."""
    if sum(S) != t: # check sum
        return False
    C = list(C) # make copy that we can change
    for y in S: # ensure S is a subcollection of C
        if y not in C:
            return False
        C.remove(y)
    return True

In [20]:
C = [1,2,3,4,5,6,7,8]
t = 20
S = [1,2,3,4,5,6]
print(subset_sum_v(C, t, S))

C = [1,2,3,4,5,6,7,8]
t = 20
S = [8, 7, 5]
print(subset_sum_v(C, t, S))

False
True


# NTM

In [30]:
import collections

def next_configs(delta, config):
    """Apply delta to configuration (state, pos, content)
    to get list of possible next configurations."""
    state, pos, content = config # give names to each part of triple
    configs = []
    symbol = content[pos]
    for (new_state, new_symbol, move) in delta(state, symbol):
        new_content = content[:pos] + new_symbol + content[pos+1:]
        new_pos = max(0, pos + move)
        if new_pos >= len(new_content):
            new_content += "-"
        config = (new_state, new_pos, new_content)
        configs.append(config)
    return configs

def M(x):
    """An NTM that replaces its input with a
    nondeterministically chosen string of the same length
    accepts if it ever guessed a 1, otherwise rejects"""
    init_config = ("q0", 0, x+"_")
    def delta(state, symbol):
        # if in halting state, no next configuration
        if state in ["qA", "qR"]:
            return []
        if symbol == "_":  # if blank, halt
            if state == "q1":
                return [("qA", "_", -1)]
            elif state == "q0":
                return [("qR", "_", -1)]
        else: 
            # write a 0 or 1 over the current symbol
            # remember if a 1 is written
            if state == "q0":
                return [ ("q0", "0", 1), ("q1", "1", 1) ]
            elif state == "q1":
                return [ ("q1", "0", 1), ("q1", "1", 1) ]
        assert False

    # do breadth-first search of configuration graph
    visited = []
    queue = collections.deque([init_config])
    while queue:
        config = queue.popleft()
        if config[0] == "qA":
            return True
        if config not in visited:
            visited.append(config)
            # get all possible next configurations; see below
            neighbors = next_configs(delta, config)
            for neighbor in neighbors:
                if neighbor not in visited:
                    queue.append(neighbor)
    return False

In [31]:
M('0101')

True

# Hamiltonian path NTM

In [35]:
import random
def P(x):
    a = random.choice([0,1])
    print(a)
P("")

0


In [41]:
import random

def ham_path_ntm(G):
    """ 'NTM' (implemented via random) for deciding HamPath. """
    V,E = G
    # guess a path p that visits each node exactly once
    p = list(V)
    random.shuffle(p)
    # verify each pair of adjacent nodes in pi shares an edge
    for i in range(len(p) - 1):
        if (p[i],p[i+1]) not in E:
            return False
    print(p)
    return True

In [42]:
V = [1,2,3,4,5]
E = [ (1,2), (2,4), (4,3), (3,5), (3,1)]
G = (V,E)
for _ in range(500):
    print(ham_path_ntm(G), end=" ")

False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False [1, 2, 4, 3, 5]
True False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False [1, 2, 4, 3, 5]
True False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False False Fals

# general exponential-time algorithm for finding $\mathsf{NP}$ witnesses

In [4]:
import itertools 

k = 7 # these can be arbitrary constants

def V_A(x,w):
    """Verifier for decision problem A."""
    raise NotImplementedError()

def binary_strings_of_length(length):
    """A nice Python way to generate all strings of a given length."""
    return map(lambda lst: ''.join(lst), itertools.product(['0', '1'], repeat=length))

def A_decider(x):
    """Exponential-time algorithm for finding witnesses."""
    n = len(x)
    for m in range(n**k + 1): # check lengths m in [0,1,...,n^c]
        for w in binary_strings_of_length(m):
            if V_A(x,w): # assumes V_A is implemented
                return True
    return False

# Hamiltonian path exponential-time deterministic algorithm

In [5]:
import itertools

def ham_path_brute_force(G):
    """Exponential-time algorithm for finding Hamiltonian paths,
    which calls the verifier on all potential witnesses."""
    V,E = G
    for p in itertools.permutations(V):
        if ham_path_verify(G, p):
            return True
    return False

In [6]:
V = [1,2,3,4,5,6]
E = [ (1,2), (2,3), (3,4), (4,1), (5,6) ]
G = (V,E)
print(ham_path_brute_force(G))

V = [1,2,3,4,5,6]
E = [ (1,2), (2,3), (3,4), (4,5), (5,6) ]
G = (V,E)
print(ham_path_brute_force(G))

False
True


# Boolean formulas

In [7]:
class Boolean_formula(object):
    """Represents a Boolean formula with AND, OR, and NOT operations."""
    def __init__(self, variable=None, op=None, left=None, right=None):
        if not ( (variable == None and op == "not" and left == None and right != None) or
                 (variable == None and op == "and" and left != None and right != None) or
                 (variable == None and op == "or"  and left != None and right != None) or
                 (variable != None and op == left == right == None) ):
            raise ValueError("Must either set variable for base case" +\
                             "or must set op to 'not', 'and', or 'or'" +\
                             "and recursive formulas left and right")
        self.variable = variable
        self.op = op
        self.left = left
        self.right = right
        if self.variable:
            self.variables = [variable]
        elif op == 'not':
            self.variables = right.variables
        elif op in ['and', 'or']:
            self.variables = list(left.variables)
            self.variables.extend(x for x in right.variables if x not in left.variables)
            self.variables.sort()

    def evaluate(self, assignment):
        """Value of this formula with given assignment, a dict mapping variable 
        names to Python booleans.

        Assignment can also be a string of bits, which will be mapped to variables 
        in alphabetical order. Boolean values are interconverted with integers to 
        make for nicer printing (0 and 1 versus True and False)"""
        if type(assignment) is str:
            assignment = dict(zip(self.variables, map(int, assignment)))
        if self.op == None:
            return assignment[self.variable]
        elif self.op == 'not':
            return int(not self.right.evaluate(assignment))
        elif self.op == 'and':
            return int(self.left.evaluate(assignment) and self.right.evaluate(assignment))
        elif self.op == 'or':
            return int(self.left.evaluate(assignment) or self.right.evaluate(assignment))
        else:
            raise ValueError("This shouldn't be reachable")

    def __repr__(self):
        if self.variable:
            return self.variable
        elif self.op == 'not':
            return '(not {})'.format(self.right)
        else:
            return '({} {} {})'.format(self.left, self.op, self.right)

    def __str__(self):
        return repr(self)

    @staticmethod
    def from_string(text):
        """Convert string that looks like a Python Boolean expression with 
        variables, e.g. "x and y or not z and (a or b)", to Boolean_formula."""
        # add plenty of whitespace to make it easy to tokenize with string.split()
        for token in ['and','or','not','(',')']:
            text = text.replace(token, ' ' + token + ' ')
        tokens = text.split()
        val_stack = []
        op_stack = []
        for token in tokens:
            if token in ['and','or','not']:
                cur_op = token
                while len(op_stack) > 0 and not precedence_greater(cur_op, op_stack[-1]):
                    process_top_op(op_stack, val_stack)
                op_stack.append(cur_op)
            elif token == '(':
                op_stack.append('(')
            elif token == ')':
                while op_stack[-1] != '(':
                    process_top_op(op_stack, val_stack)
                op_stack.pop()
            else:
                val_stack.append(Boolean_formula(variable=token))
        while len(op_stack) > 0 and not precedence_greater(cur_op, op_stack[-1]):
            process_top_op(op_stack, val_stack)
        return val_stack.pop()

def process_top_op(op_stack, val_stack):
    """Processes top operator from op_stack, popping one or two values as needed 
    from val_stack, and pushing the result back on the value stack."""
    op = op_stack.pop()
    right = val_stack.pop()
    if op == 'not':
        val_stack.append(Boolean_formula(op='not', right=right))
    elif op in ['and','or']:
        left = val_stack.pop()
        val_stack.append(Boolean_formula(op=op, left=left, right=right))

def precedence_greater(op1, op2):
    return (op2 == '(') or (op1=='not' and op2 in ['and','or']) or (op1=='and' and op2=='or')

In [16]:
formula = Boolean_formula.from_string('((x and y) or ((not z) and (a or b)))')
print(formula.evaluate("11001"))

formula = Boolean_formula.from_string('x and y or not z and (a or b)')
print(formula.evaluate("11001"))

0
0


In [10]:
formula = Boolean_formula.from_string("((x and y) or (z and (not y)))")
import itertools
num_variables = len(formula.variables)
for assignment in itertools.product(["0","1"], repeat=num_variables):
    assignment = "".join(assignment)
    value = formula.evaluate(assignment)
    print("formula value = {} on assignment {}".format(value, assignment))

formula value = 0 on assignment 000
formula value = 1 on assignment 001
formula value = 0 on assignment 010
formula value = 0 on assignment 011
formula value = 0 on assignment 100
formula value = 1 on assignment 101
formula value = 1 on assignment 110
formula value = 1 on assignment 111


In [9]:
formula = Boolean_formula.from_string("x and y or not z and (a or b)")
import itertools
num_variables = len(formula.variables)
for assignment in itertools.product(["0","1"], repeat=num_variables):
    assignment = "".join(assignment)
    value = formula.evaluate(assignment)
    print("formula value = {} on assignment {}".format(value, assignment))

formula value = 0 on assignment 00000
formula value = 0 on assignment 00001
formula value = 0 on assignment 00010
formula value = 0 on assignment 00011
formula value = 0 on assignment 00100
formula value = 0 on assignment 00101
formula value = 1 on assignment 00110
formula value = 1 on assignment 00111
formula value = 1 on assignment 01000
formula value = 0 on assignment 01001
formula value = 1 on assignment 01010
formula value = 0 on assignment 01011
formula value = 1 on assignment 01100
formula value = 0 on assignment 01101
formula value = 1 on assignment 01110
formula value = 1 on assignment 01111
formula value = 1 on assignment 10000
formula value = 0 on assignment 10001
formula value = 1 on assignment 10010
formula value = 0 on assignment 10011
formula value = 1 on assignment 10100
formula value = 0 on assignment 10101
formula value = 1 on assignment 10110
formula value = 1 on assignment 10111
formula value = 1 on assignment 11000
formula value = 0 on assignment 11001
formula valu

# polynomial-time mapping reduction of Clique to Independent-Set

In [48]:
import itertools
def reduction_from_independent_set_to_clique(G,k):
    V,E = G
    Ec = [ {u,v} for (u,v) in itertools.combinations(V,2) 
           if {u,v} not in E ]
    Gc = (V,Ec)
    return (Gc,k)

# hypothetical polynomial-time algorithm for Independent-Set,
# which calls another hypothetical polynomial-time algorithm A for Clique
def independent_set_algorithm(G,k):
    Gp,kp = reduction_from_clique_to_independent_set(G,k)
    return clique_algorithm(Gp,kp)

def clique_algorithm(G,k):
    raise NotImplementedError()

In [50]:
from pprint import pprint
V = [1,2,3,4,5,6,7]
E = [ {1,2}, {2,3}, {3,4}, {1,5}, {5,6}, {6,7}, {2,5}, {2,6}, {3,5}, {3,6}, {3,7} ]
G = (V,E)
pprint(G)
k = 4
pprint(reduction_from_independent_set_to_clique(G,k))

([1, 2, 3, 4, 5, 6, 7],
 [{1, 2},
  {2, 3},
  {3, 4},
  {1, 5},
  {5, 6},
  {6, 7},
  {2, 5},
  {2, 6},
  {3, 5},
  {3, 6},
  {3, 7}])
(([1, 2, 3, 4, 5, 6, 7],
  [{1, 3},
   {1, 4},
   {1, 6},
   {1, 7},
   {2, 4},
   {2, 7},
   {4, 5},
   {4, 6},
   {4, 7},
   {5, 7}]),
 4)


# using mapping reduction to show if $B \in \mathsf{P}$, then $A \in \mathsf{P}$

In [51]:
def f(x):
    raise NotImplementedError()
    # TODO: code for f, reducing A to B, goes here

def M_B(y):
    raise NotImplementedError()
    # TODO: code for M_B, deciding B, goes here

def M_A(x):
    """Compose reduction f from A to B, with algorithm M for B,
    to get algorithm M_A for A."""
    y = f(x)
    output = M_B(y)
    return output

# 3CNF formulas

In [11]:
class CNF(object):
    """Represents a CNF formula. Each variable is a string (e.g., "x1",
    and each clause is a tuple of strings, each either a variable
    or its negation, e.g., ("!x1", "x3", "!x4") """
    def __init__(self, clauses):
        self.variables = extract_variables(clauses)
        self.clauses = clauses
        
    def evaluate(self, assignment):
        """Value of this formula with given assignment, which is a dict mapping variable names 
        to Python booleans.
        
        Assignment can also be a string of bits, which will be mapped to variables in sorted order
        they were specified in the constructor.
        
        Boolean values are interconverted with integers to make for nicer printing 
        (0 and 1 versus True and False)"""
        if type(assignment) is str:
            assignment = dict(zip(self.variables, map(int, assignment)))
        return int(all( clause_evaluate(clause, assignment) for clause in self.clauses ))

def extract_variables(clauses):
    variables = set()
    for clause in clauses:
        for literal in clause:
            variable = literal[1:] if literal[0] == "!" else literal
            variables.add(variable)
    return sorted(list(variables))
    
def clause_evaluate(clause, assignment):
    """Value of this clause with given assignment."""
    return any( literal_evaluate(literal, assignment) for literal in clause )

def literal_evaluate(literal, assignment):
    """Value of this literal with given assignment."""
    if literal[0] == "!":
        variable = literal[1:]
        return not assignment[variable]
    else:
        variable = literal
        return assignment[variable]

In [12]:
formula = CNF([("x1", "!x2", "!x3"), ("x3", "!x5", "x6"), ("x3", "!x6", "x4"), ("x4", "x5", "x6")])
import itertools
num_variables = len(formula.variables)
for assignment in itertools.product(["0","1"], repeat=num_variables):
    assignment = "".join(assignment)
    value = formula.evaluate(assignment)
    print("formula value = {} on assignment {}".format(value, assignment))

formula value = 0 on assignment 000000
formula value = 0 on assignment 000001
formula value = 0 on assignment 000010
formula value = 0 on assignment 000011
formula value = 1 on assignment 000100
formula value = 1 on assignment 000101
formula value = 0 on assignment 000110
formula value = 1 on assignment 000111
formula value = 0 on assignment 001000
formula value = 1 on assignment 001001
formula value = 1 on assignment 001010
formula value = 1 on assignment 001011
formula value = 1 on assignment 001100
formula value = 1 on assignment 001101
formula value = 1 on assignment 001110
formula value = 1 on assignment 001111
formula value = 0 on assignment 010000
formula value = 0 on assignment 010001
formula value = 0 on assignment 010010
formula value = 0 on assignment 010011
formula value = 1 on assignment 010100
formula value = 1 on assignment 010101
formula value = 0 on assignment 010110
formula value = 1 on assignment 010111
formula value = 0 on assignment 011000
formula value = 0 on assi

# reduction of 3SAT to IndSet

In [15]:
import itertools
def reduce_3sat_to_indset(cnf_formula):
    nodes_grouped = [(('a',idx,clause[0]),
                      ('b',idx,clause[1]),
                      ('c',idx,clause[2]))
                     for (clause,idx) in zip(cnf_formula.clauses, itertools.count())]
    V = [ node for node_group in nodes_grouped for node in node_group ]
    E = []
    for (u,v) in itertools.combinations(V,2):
        add_edge = False
        (_,clause_idx1,lit1),(_,clause_idx2,lit2) = u,v
        # add edge if nodes are in same clause
        for clause in cnf_formula.clauses:
            if clause_idx1 == clause_idx2:
                add_edge = True
        if lit1 == "!"+lit2 or lit2 == "!"+lit1:
            add_edge = True
        if add_edge:
            E.append({u,v})
    k = len(cnf_formula.clauses)
    return ((V,E), k)

formula = CNF([("x1", "x1", "x2"), ("!x1", "!x2", "!x2"), ("!x1", "x2", "x2")])

from pprint import pprint
G,k = reduce_3sat_to_indset(formula)
pprint(G)

([('a', 0, 'x1'),
  ('b', 0, 'x1'),
  ('c', 0, 'x2'),
  ('a', 1, '!x1'),
  ('b', 1, '!x2'),
  ('c', 1, '!x2'),
  ('a', 2, '!x1'),
  ('b', 2, 'x2'),
  ('c', 2, 'x2')],
 [{('a', 0, 'x1'), ('b', 0, 'x1')},
  {('a', 0, 'x1'), ('c', 0, 'x2')},
  {('a', 0, 'x1'), ('a', 1, '!x1')},
  {('a', 0, 'x1'), ('a', 2, '!x1')},
  {('b', 0, 'x1'), ('c', 0, 'x2')},
  {('a', 1, '!x1'), ('b', 0, 'x1')},
  {('a', 2, '!x1'), ('b', 0, 'x1')},
  {('c', 0, 'x2'), ('b', 1, '!x2')},
  {('c', 1, '!x2'), ('c', 0, 'x2')},
  {('a', 1, '!x1'), ('b', 1, '!x2')},
  {('a', 1, '!x1'), ('c', 1, '!x2')},
  {('c', 1, '!x2'), ('b', 1, '!x2')},
  {('b', 2, 'x2'), ('b', 1, '!x2')},
  {('c', 2, 'x2'), ('b', 1, '!x2')},
  {('c', 1, '!x2'), ('b', 2, 'x2')},
  {('c', 2, 'x2'), ('c', 1, '!x2')},
  {('a', 2, '!x1'), ('b', 2, 'x2')},
  {('a', 2, '!x1'), ('c', 2, 'x2')},
  {('c', 2, 'x2'), ('b', 2, 'x2')}])


In [16]:
# this code shows how one would use a hypothetical algorithm for independent set, 
# together with reduce_3sat_to_independent, to create an algorithm for 3SAT

import itertools
def independent_set(G,k):
    """This is an exponential-time algorithm for IndSet, just to 
    make the code below work properly. But if we could replace this 
    with a polynomial-time algorithm, then the function three_sat
    would also be polynomial-time."""
    V,E = G
    for S in itertools.combinations(V,k):
        is_IS = True
        for (u,v) in itertools.combinations(S, 2):
            if {u,v} in E:
                is_IS = False
                break
        if is_IS:
            return True
    return False
    
def three_sat(formula):
    G,k = reduce_3sat_to_indset(formula)
    return independent_set(G,k)

formula = CNF([("x1", "x1", "x2"), ("!x1", "!x2", "!x2"), ("!x1", "x2", "x2")])
print("is formula satisfiable? {}".format(three_sat(formula)))

formula = CNF([("x1", "x1", "x2"), ("x1", "x1", "!x2"), ("!x1", "x2", "x2"), ("!x1", "!x1", "!x2")])
print("is formula satisfiable? {}".format(three_sat(formula)))

is formula satisfiable? True
is formula satisfiable? False


# enumerators

In [56]:
import itertools
def primes_enumerator():
    """Iterate over all prime numbers."""
    for n in itertools.count(): #iterates over all natural numbers
        if is_prime(n):
            yield n

In [57]:
# print first 100 numbers returned from primes_enumerator()
for (p, _) in zip(primes_enumerator(), range(100)):
    print(p, end=" ")

2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97 101 103 107 109 113 127 131 137 139 149 151 157 163 167 173 179 181 191 193 197 199 211 223 227 229 233 239 241 251 257 263 269 271 277 281 283 293 307 311 313 317 331 337 347 349 353 359 367 373 379 383 389 397 401 409 419 421 431 433 439 443 449 457 461 463 467 479 487 491 499 503 509 521 523 541 

# converting enumerator to recognizer

In [58]:
def create_recognizer_from_enumerator(enumerator):
    def recognizer(input_to_recognizer):
        for enumerated_output in enumerator():
            if enumerated_output == input_to_recognizer:
                return True
        return False  # if it has a finite language it might halt
    return recognizer

In [59]:
primes_recognizer = create_recognizer_from_enumerator(primes_enumerator)
print(primes_recognizer(2))  # prints True
print(primes_recognizer(3))  # prints True
print(primes_recognizer(5))  # prints True
print(primes_recognizer(7))  # prints True
print(primes_recognizer(9))  # runs forever

True
True
True
True


KeyboardInterrupt: 

# converting recognizer to enumerator

In [61]:
import sys, itertools, threading

# for python 3 include the next line
from queue import Queue 

# for python 2 include the next line
# from Queue import Queue 

def binary_strings():
    """Enumerates all binary strings in lexicographical order."""
    for length in itertools.count():
        for s in itertools.product(["0","1"], repeat=length):
            yield "".join(s)

#WARNING: this will run forever and consume processor and memory and have to be killed
def enumerator_from_recognizer(recognizer, inputs=binary_strings()):
    """Given recognizer, a function that defines a language by returning True if 
    its input string is in the language and otherwise either returning False 
    or looping, this function enumerates over strings in the language.

    inputs is iterator yielding valid inputs to recognizer. Default is binary 
    strings in lexicographical order."""
    # define queue to put outputs (and its associated input) into
    outputs_queue = Queue()
    def compute_output_and_add_to_queue(input_string):
        output = recognizer(input_string)
        if output == True:
            outputs_queue.put((input_string, output))

    # run recognizer on all possible inputs and add outputs to outputs_queue
    recognizer_threads = []
    def start_all_recognizer_threads():
        for input_string in inputs:
            thread = threading.Thread(target=compute_output_and_add_to_queue, 
                                      args=[input_string])
            thread.daemon = True
            thread.start()
            recognizer_threads.append(thread)
        # if finite number of inputs, wait until all threads have stopped
        # and then add a new item to the queue to indicate all inputs are done
        for thread in recognizer_threads:
            thread.join()
        outputs_queue.put((None,None))

    starter_thread = threading.Thread(target=start_all_recognizer_threads)
    starter_thread.daemon = True # ensures thread is killed when main thread stops
    starter_thread.start()

    # search for True output in outputs_queue
    while True:
        (input_string, output) = outputs_queue.get()
        if output == None:
            break
        yield input_string
        

#WARNING: this will run forever and consume processor and memory and have to be killed
def create_enumerator_from_recognizer(recognizer, inputs=binary_strings()):
    """Given recognizer, a function that defines a language by returning True 
    if its input string is in the language and otherwise either returning False 
    or looping, it returns an iterator that iterates over strings in the language.

    inputs is iterator yielding valid inputs to recognizer. Default is binary 
    strings in lexicographical order."""
    def enumerator():
        return enumerator_from_recognizer(recognizer, inputs)
    return enumerator

In [66]:
# for python 2 include the next line
# from future_builtins import zip

import itertools
enumerate_primes_created_from_recognizer = create_enumerator_from_recognizer(is_prime, inputs=itertools.count())

#WARNING: this will run forever and consume processor and memory and have to be killed
for i,p in zip(itertools.count(), enumerate_primes_created_from_recognizer()):
    print("{}'th prime is {}".format(i, p))
    if i > 10: 
        break

0'th prime is 2
1'th prime is 3
2'th prime is 5
3'th prime is 7
4'th prime is 11
5'th prime is 13
6'th prime is 17
7'th prime is 19
8'th prime is 23
9'th prime is 29
10'th prime is 31
11'th prime is 37


In [65]:
import itertools
enumerate_primes = create_enumerator_from_recognizer(is_prime, inputs=range(100))

for i,p in zip(itertools.count(), enumerate_primes()):
    print("{}'th prime is {}".format(i, p))

0'th prime is 2
1'th prime is 3
2'th prime is 5
3'th prime is 7
4'th prime is 11
5'th prime is 13
6'th prime is 17
7'th prime is 19
8'th prime is 23
9'th prime is 29
10'th prime is 31
11'th prime is 37
12'th prime is 41
13'th prime is 43
14'th prime is 47
15'th prime is 53
16'th prime is 59
17'th prime is 61
18'th prime is 67
19'th prime is 71
20'th prime is 73
21'th prime is 79
22'th prime is 83
23'th prime is 89
24'th prime is 97


# $H_\mathsf{TM}$ is Turing-recognizable (via a recognizer)

In [1]:
def halt_recognizer(M_src,w):
    """M is the source code of a Python function named M,
    and w is an input to M."""
    namespace = {}
    exec(M_src, namespace) # this defines the function M so we can execute it
    M = namespace['M'] # this is needed to find the variable M set when the code in M_src executes (not needed in Python 2)
    M(w)        # this actually executes the function on input w
    return True # this is only reached if M(w) halts

In [2]:
M_src = """
def M(w):
    '''Indicates if x has more than 5 occurences of the symbol "a".'''
    count = 0
    for c in w:
        if c == "a":
            count += 1
    return count > 5
"""
w = "abcabcaaabcaa"
halt_recognizer(M_src, w) # will halt since M always halts

True

In [3]:
M_src = """
def M(w):
    '''Has a potential infinite loop.'''
    count = 0
    while count < 5:
        c = w[0]
        if c == "a":
            count += 1
    return count > 5
"""
w = "abcabcaaabcaa"
print(halt_recognizer(M_src, w)) # will halt since w[0] == "a"

True


In [4]:
M_src = """
def M(w):
    '''Has a potential infinite loop.'''
    count = 0
    while count < 5:
        c = w[0]
        if c == "a":
            count += 1
    return count > 5
"""
# WARNING: this runs forever and will have to be killed
w = "bcabcaaabcaa"
print(halt_recognizer(M_src, w)) # won't halt since w[0] != "a"

KeyboardInterrupt: 

# converting between Python programs and source code

In [5]:
def M(w):
    """Indicates if |w| > 3"""
    return len(w) > 3

print('M("0101") = {}'.format(M("0101")))

# use inspect.getsource to convert function to source code
import inspect
M_src = inspect.getsource(M)
print("source code of M:\n{}".format(M_src))

M_src = M_src.replace("len(w) > 3", "len(w) > 5")

# use exec to convert source code to function
namespace = {}
exec(M_src, namespace)
M = namespace['M']

print('M("0101") = {}'.format(M("0101")))
print("source code of M:\n{}".format(M_src))

M("0101") = True
source code of M:
def M(w):
    """Indicates if |w| > 3"""
    return len(w) > 3

M("0101") = False
source code of M:
def M(w):
    """Indicates if |w| > 3"""
    return len(w) > 5



# halting problem recognizer as Python function object (instead of string with source code)

In [6]:
def halt_recognizer(M,w):
    """M is a Python function, and w is an input to M."""
    M(w)        # this executes the function M on input w
    return True # this is only reached if M(w) halts

In [7]:
def M(w):
    """Indicates if x has > 5 occurences of the symbol "a"."""
    count = 0
    for c in w:
        if c == "a":
            count += 1
    return count > 5

w = "abcabcaaabcaa"
halt_recognizer(M, w) # will halt since M always halts

True

In [9]:
def M(w):
    """Has a potential infinite loop."""
    count = 0
    while count < 5:
        c = w[0]
        if c == "a":
            count += 1
    return count > 5

w = "abcabcaaabcaa"
print(halt_recognizer(M, w)) # will halt since w[0] == "a"

True


In [77]:
def M(w):
    """Has a potential infinite loop."""
    count = 0
    while count < 5:
        c = w[0]
        if c == "a":
            count += 1
    return count > 5

# WARNING: will not halt!
# After running this, you'll need to kill the interpreter
w = "bcabcaaabcaa"
print(halt_recognizer(M, w)) # won't halt since w[0] != "a"

KeyboardInterrupt: 

# converting recognizers for $A$ and $\overline{A}$ to decider for $A$

In [85]:
# for python 3 include the next line
from queue import Queue 

# for python 2 include the next line
# from Queue import Queue 

import threading

def decider_from_recognizers(recognizer, comp_recognizer, w):
    """recognizer is function recognizing some language A, and
    comp_recognizer is function recognizing complement of A."""
    outputs_queue = Queue()
    
    def recognizer_add_to_queue():
        output = recognizer(w)
        if output == True:
            outputs_queue.put("w in A")
    
    def comp_recognizer_add_to_queue():
        output = comp_recognizer(w)
        if output == True:
            outputs_queue.put("w not in A")
    
    t1 = threading.Thread(target = recognizer_add_to_queue)
    t2 = threading.Thread(target = comp_recognizer_add_to_queue)
    t1.daemon = t2.daemon = True
    t1.start()
    t2.start()
    
    # exactly one of the threads will put a message in the queue
    message = outputs_queue.get()
    if message == "w in A":
        return True
    elif message == "w not in A":
        return False
    else:
        raise AssertionError("should not be reachable")
        
def create_decider_from_recognizers(recognizer, comp_recognizer):
    def decider(w):
        return decider_from_recognizers(recognizer, comp_recognizer, w)
    return decider

In [ ]:
def loop():
    """Loop forever."""
    while True:
        pass

def prime_recognizer(n):
    """Check if n is prime."""
    if n < 2: # 0 and 1 are not primes
        loop()
    for x in range(2, int(n**0.5)+1):
        if n % x == 0:
            loop()
    return True

def composite_recognizer(n):
    if n < 2: # 0 and 1 are not primes
        return True
    for x in xrange(2, int(n**0.5)+1):
        if n % x == 0:
            return True
    loop()

#WARNING: will continue to consume resources after it halts
# because it starts up threads that run forever
prime_decider_from_recognizers = create_decider_from_recognizers(prime_recognizer, composite_recognizer)
for n in range(2,20):
    print("{:2} is prime?  {}".format(n, prime_decider_from_recognizers(n)))

# converting enumerators for $A$ and $\overline{A}$ to decider for $A$


In [83]:
# for python 2 include next line
# from future_builtins import zip

def decider_from_enumerators(enumerator, comp_enumerator, w):
    """enumerator is Python generator enumerating language A, and 
    comp_enumerator is generator enumerating complement of A."""
    for x,y in zip(enumerator(), comp_enumerator()):
        if x == w:
            return True
        if y == w:
            return False

def create_decider_from_enumerators(enumerator, comp_enumerator):
    def decider(w):
        return decider_from_enumerators(enumerator, comp_enumerator, w)
    return decider

In [84]:
import itertools
def primes_enumerator():
    """Iterate over all prime numbers."""
    for n in itertools.count():
        if is_prime(n):
            yield n

def composites_enumerator():
    """Iterate over all composite numbers."""
    for n in itertools.count():
        if not is_prime(n):
            yield n

prime_decider = create_decider_from_enumerators(primes_enumerator, composites_enumerator)
for n in range(2,20):
    print("{:2} is prime?  {}".format(n, prime_decider(n)))

 2 is prime?  True
 3 is prime?  True
 4 is prime?  False
 5 is prime?  True
 6 is prime?  False
 7 is prime?  True
 8 is prime?  False
 9 is prime?  False
10 is prime?  False
11 is prime?  True
12 is prime?  False
13 is prime?  True
14 is prime?  False
15 is prime?  False
16 is prime?  False
17 is prime?  True
18 is prime?  False
19 is prime?  True


# reduction of $H_\mathsf{TM}$ to $A_\mathsf{TM}$

In [79]:
def A(M,w):
    """Function that supposedly decides whether M(w) accepts."""
    raise NotImplementedError()

def opposite(M):
    """Given M, a Python function returning a Boolean, returns 
    Python function M_O that acts like M, but returns the 
    opposite value (but also loops whenever M loops)."""
    def M_O(w):
        return not M(w)
    return M_O

def H(M,w):
    """Decider for the halting problem that works assuming A works,
    giving a contradiction and proving A cannot be implemented."""
    M_O = opposite(M)
    return A(M,w) or A(M_O,w)

In [80]:
def M(x):
    count = 0
    for c in x:
        if c == "a":
            count += 1
    return count > 5

M_O = opposite(M)

print("M('abababab')   = {}".format(M('abababab')))
print("M_O('abababab') = {}".format(M_O('abababab')))

M('abababab')   = False
M_O('abababab') = True


# reduction of $H_\mathsf{TM}$ to $H_\mathsf{TM}^\varepsilon$

In [77]:
def H_e(M):
    """Function that supposedly decides whether M("") halts."""
    raise NotImplementedError()

def H(M,w):
    """Decider for HALT_TM that works assuming H_e works,
    giving a contradiction and proving H_e cannot be implemented."""
    def N_Mw(x):
        """Halts on every string if M halts on w, and loops on 
        every string if M loops on w."""
        M(w)
    return H_e(N_Mw)

# reduction of $H_\mathsf{TM}$ to $E_\mathsf{TM}$

In [76]:
def E(M,w):
    """Function that supposedly decides whether L(M) is empty."""
    raise NotImplementedError()

def H(M,w):
    """Decider for HALT_TM that works assuming E works,
    giving a contradiction and proving E cannot be implemented."""
    def N_Mw(x):
        """Accepts every string if M halts on w, and loops on 
        every string if M loops on w."""
        M(w)
        return True
    return not E(N_Mw)

# $H_\mathsf{TM}$ is undecidable

In [82]:
def H(M,w):
    """Function that supposedly decides whether M(w) halts."""
    raise NotImplementedError()  

def D(M):
    """ "Diagonalizing" function; given a function M, if M halts 
    when given itself as input, run forever; otherwise halt."""
    if H(M,M):
        while True:  # loop forever
            pass
    else:
        return True  # halt

D(D) # result not well-defined! H cannot be implemented